In [1]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup

In [2]:
def getAmazonSearchQuery(searchQuery, pageNumber, header):
    urlQuery = "https://www.amazon.com.br/s?k=" + searchQuery + "&page="
    pageRequest = req.get(urlQuery, headers = header)

    if pageRequest.status_code == 200:
        return pageRequest
    else:
        return "Error while making the query search on Amazon."

In [3]:
def getProductReviews(asin, pageNumber, header, cookies):
    urlQuery = "https://www.amazon.com.br/product-reviews/" + asin + "/reviewerType=all_reviews?sortBy=recent&pageNumber=" + str(pageNumber)
    pageRequest = req.get(urlQuery, headers = header, cookies = cookies)

    if pageRequest.status_code == 200:
        return pageRequest
    else:
        return "Error while getting the product's reviews on Amazon."

In [4]:
productsNames = list()
productsLinks = list()
productsAsin = list()

searchQuery = "Fone de ouvido JBL"
searchQuery = searchQuery.replace(" ", "+")

In [9]:
headerResponse = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}

responseGet = getAmazonSearchQuery(searchQuery, 1, headerResponse)

cookiesResponse = responseGet.cookies
contentResponse = responseGet.content

In [10]:
soupGet = BeautifulSoup(contentResponse)

for product in soupGet.findAll("h2", {"class": "a-size-mini a-spacing-none a-color-base s-line-clamp-4"}):
    productsNames.append(product.text)

    productHref = product.find("a", {"class": "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})
    productLink = "https://www.amazon.com.br/" + productHref['href']
    productsLinks.append(productLink)

    indexDp = productLink.index("dp/")
    indexRef = productLink.index("/ref")
    productId = productLink[indexDp + 3:indexRef]

    productsAsin.append(productId)

In [19]:
reviewsTitles = list()
reviewsStars = list()
reviewsText = list()

responseReviewsGet = getProductReviews(productsAsin[0], 1, headerResponse, cookiesResponse)
contentResponseReviews = responseReviewsGet.content

In [20]:
soupReviewsGet = BeautifulSoup(contentResponseReviews)

for review in soupReviewsGet.findAll("a", {"data-hook": "review-title"}):
    reviewsTitles.append(review.text)

for review in soupReviewsGet.findAll("div", {"id": "cm_cr-review_list"}):
    for title in review.findAll("a", {"data-hook": "review-title"}):
        reviewsTitles.append(title.text.replace("\n", ""))
    for body in review.findAll("a", {"data-hook": "review-body"}):
        reviewsText.append(body.text)
    for stars in review.findAll("i", {"data-hook": "review-star-rating"}):
        reviewsStars.append(stars.text)

In [27]:
reviewsFinal = {"Title": reviewsTitles}

reviewsData = pd.DataFrame.from_dict(reviewsFinal)
pd.set_option("max_colwidth", 8000)
reviewsData

,Title
0,"\nFone Excelente, esse sim entrega a qualidade de som que eu tanto esperava 🤍🧡\n"
1,\nMaravilhos - super recomendo!!\n
2,\nQualidade do som baixa\n
3,\ntop\n
4,\nbom\n
5,\nO queridinho!\n
6,\nFone\n
7,\nOtimo\n
8,\nÓtimo custo benefício\n
9,\nFone Tune 10\n
